In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CdYCXjBLXkB2aYoCaCbqOBAVtIwTY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--8d9b72fc-1c8f-47d1-b3c8-9fe5ee9d523b-0' usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm.invoke(question)

In [ ]:
response.content

'대한민국의 수도는 서울입니다.'

In [ ]:
from typing import Annotated ,Literal ,TypedDict

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage ,SystemMessage
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import END ,START,StateGraph
from langgraph.graph.message import add_messages

embeddings=OpenAIEmbeddings()

model_low_temp =ChatOpenAI(model='gpt-4o-mini' ,temperature=0.1)
model_high_temp=ChatOpenAI(model ='gpt-4o-mini' ,temperature=0.7)

class State(TypedDict):
    messages:Annotated[list ,add_messages]
    user_query :str
    domain:Literal['records' ,'insurance']
    documents:list[Document]
    answer:str
    
class Input(TypedDict):
    user_query:str
    
class Output(TypedDict):
    documents:list[Document]
    answer:str
    
medical_records_store=InMemoryVectorStore.from_documents([],embeddings)
medical_records_retriever=medical_records_store.as_retriever()

insurance_faqs_store=InMemoryVectorStore.from_documents([],embeddings)
insurance_faqs_retriever=insurance_faqs_store.as_retriever()

router_prompt=SystemMessage(
    '''
    사용자 문의를 어느 도메인으로 라우팅할지 결정하세요.선택할 수 있는 두 가지 도메인은 다음과 같습니다.
    -records: 진단,치료,처방과 같은 환자의 의료 기록을 포함합니다.
    -insurance: 보험 정책,청구,보장에 대한 자주 묻는 질문을 포함합니다.
    도메인 이름만 출력하세요.
    '''
)

def router_node(state:State) ->State:
    user_messages=HumanMessage(state['user_query'])
    messages =[router_prompt, *state['messages'] ,user_messages]
    res= model_low_temp.invoke(messages)
    return {
        'domain':res.content,
        'messages':[user_messages ,res]
    }
    
def pick_retriever(
    state:State
) ->Literal['retrieve_medical_records' ,'retrieve_insurance_faqs']:
    if state['domain']=='records':
        return 'retrieve_medical_records'
    else:
        return 'retrieve_insurance_faqs'
    
def retrieve_medical_records(state:State)->State:
    documents=medical_records_retriever.invoke(state['user_query'])
    return {
        'documents':documents
    }
    
def retrieve_insurance_faqs(state:State) ->State:
    documents=insurance_faqs_retriever.invoke(state['user_query'])
    return {
        'documents':documents
    }

medical_records_prompt=SystemMessage(
    '당신은 유능한 의료 챗봇입니다. 진단,치료,처방과 같은 환자의 의료 기록을 기반으로 질문에 답하세요.'

)

insurance_faqs_prompt= SystemMessage(
    '당신은 유능한 의료 보험 챗봇입니다. 보험 정책, 청구 및 보장에 대한 자주 묻는 질문에 답하세요.'
)

def generate_answer(state: State)->State:
    if state['domain']=='records':
        prompt=medical_records_prompt
    else:
        prompt=insurance_faqs_prompt
    
    messages=[
        prompt,
        *state['messages'],
        HumanMessage(f"Documents: {state['documents']}")
    ]
    res=model_high_temp.invoke(messages)
    return {
        'answer':res.content,
        'messages':res
    }
    
builder=StateGraph(State ,input=Input ,output=Output)
builder.add_node('router',router_node)
builder.add_node('retireve_medical_records' ,retrieve_medical_records)
builder.add_node('retrieve_insrance_faqs' ,retrieve_insurance_faqs)
builder.add_node('generate_answer' ,generate_answer)
builder.add_edge(START ,'router')
builder.add_conditional_edges('router' ,pick_retriever)
builder.add_edge('retrieve_medical_records' ,'generate_answer')
builder.add_edge('retrieve_insurance_faqs' ,'generate_answer')
builder.add_edge('generate_answer' ,END)

graph=builder.compile()